In [1]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import colorsys
import numpy as np
import cv2
import tensorflow as tf
from copy import deepcopy

In [7]:
class BoundBox:
    def __init__(self, xmin, ymin, xmax, ymax, objness = None, classes = None):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax

        self.objness = objness
        self.classes = classes

        self.label = -1
        self.score = -1

    def get_label(self):
        if self.label == -1:
            self.label = np.argmax(self.classes)
        return self.label

    def get_score(self):
        if self.score == -1:
            self.score = self.classes[self.get_label()]
        return self.score
    

def _interval_overlap(interval_a, interval_b):
    x1, x2 = interval_a
    x3, x4 = interval_b

    if x3 < x1:
        if x4 < x1:
            return 0
        else:
            return min(x2, x4) - x1
    else:
        if x2 < x3:
            return 0
        else:
            return min(x2, x4) - x3

def _sigmoid(x):
    return 1. / (1. + np.exp(-x))


def bbox_iou(box1, box2):
    intersect_w = _interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
    intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])

    intersect = intersect_w * intersect_h

    w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
    w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin

    union = w1*h1 + w2*h2 - intersect

    return float(intersect) / union


def preprocess_input(image_pil, net_h, net_w):
    image = np.asarray(image_pil)
    new_h, new_w, _ = image.shape

    # (1) - starts  
    if (float(net_w)/new_w) < (float(net_h)/new_h):
        new_h = (new_h * net_w)/new_w
        new_w = net_w
    else:
        new_w = (new_w * net_h)/new_h
        new_h = net_h 
    # (1) - ends
    
    new_w = int(new_w)
    new_h = int(new_h)

    # (2) - starts
    resized = cv2.resize(image/255., (int(new_w), int(new_h)))
    # (2) - ends

    # (3) - starts
    new_image = np.ones((net_h, net_w, 3)) * 0.5
    new_image[int((net_h-new_h)//2):int((net_h+new_h)//2), int((net_w-new_w)//2):int((net_w+new_w)//2), :] = resized
    # (3) - ends

    # (4) - starts
    new_image = np.expand_dims(new_image, 0)
    # (4) - ends

    # The new image maintains aspect ratio - (1), normalizes pixel values (faster for neural networks) - (2), 
    # Adds padding (letterboxing) - (3), and expands batch dimension to make it compatible with deep learning models - (4) !!!

    return new_image


def decode_netout(netout_, obj_thresh, anchors_, image_h, image_w, net_h, net_w):
    netout_all = deepcopy(netout_)
    boxes_all = []
    for i in range(len(netout_all)):
      netout = netout_all[i][0]
      anchors = anchors_[i]

      grid_h, grid_w = netout.shape[:2]
      nb_box = 3
      netout = netout.reshape((grid_h, grid_w, nb_box, -1))
      nb_class = netout.shape[-1] - 5

      boxes = []

      netout[..., :2]  = _sigmoid(netout[..., :2])
      netout[..., 4:]  = _sigmoid(netout[..., 4:])
      netout[..., 5:]  = netout[..., 4][..., np.newaxis] * netout[..., 5:]
      netout[..., 5:] *= netout[..., 5:] > obj_thresh

      for i in range(grid_h*grid_w):
          row = i // grid_w
          col = i % grid_w

          for b in range(nb_box):
              # 4th element is objectness score
              objectness = netout[row][col][b][4]
              #objectness = netout[..., :4]
              # last elements are class probabilities
              classes = netout[row][col][b][5:]

              if((classes <= obj_thresh).all()): continue

              # first 4 elements are x, y, w, and h
              x, y, w, h = netout[row][col][b][:4]

              x = (col + x) / grid_w # center position, unit: image width
              y = (row + y) / grid_h # center position, unit: image height
              w = anchors[b][0] * np.exp(w) / net_w # unit: image width
              h = anchors[b][1] * np.exp(h) / net_h # unit: image height

              box = BoundBox(x-w/2, y-h/2, x+w/2, y+h/2, objectness, classes)
              #box = BoundBox(x-w/2, y-h/2, x+w/2, y+h/2, None, classes)

              boxes.append(box)

      boxes_all += boxes

    # Correct boxes
    boxes_all = correct_yolo_boxes(boxes_all, image_h, image_w, net_h, net_w)

    return boxes_all

def correct_yolo_boxes(boxes_, image_h, image_w, net_h, net_w):
    boxes = deepcopy(boxes_)
    if (float(net_w)/image_w) < (float(net_h)/image_h):
        new_w = net_w
        new_h = (image_h*net_w)/image_w
    else:
        new_h = net_w
        new_w = (image_w*net_h)/image_h

    for i in range(len(boxes)):
        x_offset, x_scale = (net_w - new_w)/2./net_w, float(new_w)/net_w
        y_offset, y_scale = (net_h - new_h)/2./net_h, float(new_h)/net_h

        boxes[i].xmin = int((boxes[i].xmin - x_offset) / x_scale * image_w)
        boxes[i].xmax = int((boxes[i].xmax - x_offset) / x_scale * image_w)
        boxes[i].ymin = int((boxes[i].ymin - y_offset) / y_scale * image_h)
        boxes[i].ymax = int((boxes[i].ymax - y_offset) / y_scale * image_h)
    return boxes

def do_nms(boxes_, nms_thresh, obj_thresh):
    boxes = deepcopy(boxes_)
    if len(boxes) > 0:
        num_class = len(boxes[0].classes)
    else:
        return

    for c in range(num_class):
        sorted_indices = np.argsort([-box.classes[c] for box in boxes])

        for i in range(len(sorted_indices)):
            index_i = sorted_indices[i]

            if boxes[index_i].classes[c] == 0: continue # Skip if already suppressed

            for j in range(i+1, len(sorted_indices)):
                index_j = sorted_indices[j]

                if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_thresh:
                    boxes[index_j].classes[c] = 0 # Suppress lower-confidence boxes

    new_boxes = []
    for box in boxes:
        label = -1

        for i in range(num_class):
            if box.classes[i] > obj_thresh:
                label = i
                # print("{}: {}, ({}, {})".format(labels[i], box.classes[i]*100, box.xmin, box.ymin))
                box.label = label
                box.score = box.classes[i]
                new_boxes.append(box)

    return new_boxes


In [8]:
def draw_boxes(image_, boxes, labels):
    image = image_.copy()
    image_w, image_h = image.size
    font = ImageFont.truetype(font='/Users/AshishR_T/Library/Fonts/Monogram Font.ttf',
                    size=np.floor(3e-2 * image_h + 0.5).astype('int32'))
    thickness = (image_w + image_h) // 300

    # Generate colors for drawing bounding boxes.
    hsv_tuples = [(x / len(labels), 1., 1.)
                  for x in range(len(labels))]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(
        map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))
    np.random.seed(10101)  # Fixed seed for consistent colors across runs.
    np.random.shuffle(colors)  # Shuffle colors to decorrelate adjacent classes.
    np.random.seed(None)  # Reset seed to default.

    for i, box in reversed(list(enumerate(boxes))):
        c = box.get_label()
        predicted_class = labels[c]
        score = box.get_score()
        top, left, bottom, right = box.ymin, box.xmin, box.ymax, box.xmax

        label = '{} {:.2f}'.format(predicted_class, score)
        draw = ImageDraw.Draw(image)
        label_size = draw.textbbox((0,0),label, font)
        label_size = (label_size[2], label_size[3])

        top = max(0, np.floor(top + 0.5).astype('int32'))
        left = max(0, np.floor(left + 0.5).astype('int32'))
        bottom = min(image_h, np.floor(bottom + 0.5).astype('int32'))
        right = min(image_w, np.floor(right + 0.5).astype('int32'))
        print(label, (left, top), (right, bottom))

        if top - label_size[1] >= 0:
            text_origin = np.array([left, top - label_size[1]])
        else:
            text_origin = np.array([left, top + 1])

        if right > left and bottom > top:
          # proceed to draw the box
          draw.rectangle([left, top, right, bottom], outline=colors[c], width=thickness)
        else:
          print(f"Invalid box: {(left, top, right, bottom)}")

        draw.text(text_origin, label, fill=(0, 0, 0), font=font)
        #draw.text(text_origin, label, fill=(0, 0, 0))
        del draw
    return image

In [9]:
#Yolo v3 model (pre-trained)
model_path = 'YOLO_model.h5'
darknet = tf.keras.models.load_model(model_path, compile = False)

/Users/AshishR_T/miniconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
2025-02-23 19:16:56.027882: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-02-23 19:16:56.027924: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-02-23 19:16:56.027931: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-02-23 19:16:56.027953: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-23 19:16:56.027966: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id:

In [10]:
labels = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", \
              "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", \
              "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", \
              "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", \
              "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", \
              "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", \
              "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", \
              "chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse", \
              "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", \
              "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]

In [11]:
anchors = [[[116,90], [156,198], [373,326]], [[30,61], [62,45], [59,119]], [[10,13], [16,30], [33,23]]]

In [12]:
def detect_image(image_pil, obj_thresh = 0.4, nms_thresh = 0.45, darknet = darknet, net_h = 416, net_w = 416, anchors = anchors, labels = labels):
  # Preprocessing
  image_w, image_h = image_pil.size
  new_image = preprocess_input(image_pil, net_h, net_w)

  # DarkNet
  yolo_outputs = darknet.predict(new_image)

  # Decode the output of the network
  boxes = decode_netout(yolo_outputs, obj_thresh, anchors, image_h, image_w, net_h, net_w)

  # Suppress non-maximal boxes
  boxes = do_nms(boxes, nms_thresh, obj_thresh)

  # Draw bounding boxes on the image using labels
  image_detect = draw_boxes(image_pil, boxes, labels)

  return image_detect

In [14]:
def capture_frame(cap):
    ret, frame = cap.read()
    if not ret:
        return None
    # Convert frame from BGR (OpenCV) to RGB (PIL)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_pil = Image.fromarray(frame_rgb)
    return image_pil

def run_webcam_detection():
    cap = cv2.VideoCapture(0)  # Open the default webcam
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        image_pil = capture_frame(cap)
        if image_pil is None:
            break

        # Use your YOLO detection function
        result_pil = detect_image(image_pil)
        
        # Convert the result (PIL image) back to OpenCV format for display
        result_np = np.array(result_pil)
        result_bgr = cv2.cvtColor(result_np, cv2.COLOR_RGB2BGR)

        cv2.imshow("Webcam Object Detection", result_bgr)

        # Exit when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

run_webcam_detection()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


[ WARN:0@36.993] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


person 1.00 (553, 144) (1730, 1066)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


2025-02-23 19:17:02.862 python[37957:1388621] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-23 19:17:02.862 python[37957:1388621] +[IMKInputSession subclass]: chose IMKInputSession_Modern


person 1.00 (555, 141) (1725, 1070)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
person 1.00 (520, 158) (1752, 1080)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
person 1.00 (506, 167) (1779, 1080)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
person 1.00 (510, 173) (1772, 1080)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
person 1.00 (526, 175) (1749, 1080)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
person 1.00 (534, 177) (1739, 1080)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
toothbrush 0.56 (1145, 620) (1325, 714)
person 1.00 (541, 190) (1723, 1080)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
person 1.00 (537, 181) (1734, 1080)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
person 1.00 (538, 173) (1741, 1080)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
person 1.00 (541, 176) (1737, 1080)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
person 1.00 (538, 176) (1740, 1080)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
person 1.00 (524, 211) (1719, 1080)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
person 1.00 (554, 258) (1648, 1080)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/ste

: 